In [2]:
import os
from datetime import datetime, timedelta
import pandas as pd
from zeep import Client
import loadenv
import db
from db import get_connection
import psycopg2.extras
import logging
import sys

logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [10]:
def parse_and_insert_sheet(df, sheet_name, col_name):
    logger.info(f"Parse sheet {sheet_name}")

    stand = df[sheet_name].iloc[1, 0]

    datenbestand = datetime.strptime(stand, 'Stand: %d.%m.%Y %H:%M:%S')

    nan_value = float("NaN")
    data = df[sheet_name].iloc[4:,2:].replace("", nan_value).dropna(how='all', axis=1)
    
    col = 0
    for d in data.loc[data.index[0]]:
        if isinstance(d, str) and d != '':
            try:
                parsed = datetime.strptime(d, '%d.%m.%Y')

                data.iat[0, col] = parsed
            except ValueError as _:
                pass

        col += 1

    # data[2] = data[2].apply(lambda x: "{:05d}".format(x) if isinstance(x, int) else x)

    entries = []
    for r_idx in range(1, data.shape[0]):
        print(data.iloc[r_idx, 0])
        for c_idx in range(1, data.shape[1]):
            val = None if data.iloc[r_idx, c_idx] == '' else data.iloc[r_idx, c_idx]
            entry = {
                'datenbestand': datenbestand,
                'id': "{:05d}".format(data.iloc[r_idx, 0]),
                'ts': data.iloc[0, c_idx],
                'val': val
            }
            entries.append(entry)
            
    return entries

#     psycopg2.extras.execute_values(
#         cur,
#         f"""INSERT INTO 
#             rki_incidence_excel (datenbestand, ags, timestamp, \"{col_name}\") 
#             VALUES %s ON CONFLICT ON CONSTRAINT rki_incidence_excel_pk DO
#             UPDATE SET
#                 \"{col_name}\" = EXCLUDED.\"{col_name}\",
#                 updated_at = NOW();
#         """,
#         entries,
#         template='(%(datenbestand)s, %(id)s, %(ts)s, %(val)s)',
#         page_size=500
#     )
#     conn.commit()

    return True


def parse_and_insert(fp: str) -> bool:
    df = pd.read_excel(fp, sheet_name=['LK_7-Tage-Inzidenz', 'LK_7-Tage-Fallzahlen'], header=None,
                       na_filter=False, engine="openpyxl")

    parse_and_insert_sheet(df=df, sheet_name='LK_7-Tage-Inzidenz', col_name='7_day_incidence')
    parse_and_insert_sheet(df=df, sheet_name='LK_7-Tage-Fallzahlen', col_name='7_day_cases')

    return True

fp = './Fallzahlen_Kum_Tab.xlsm'
df = pd.read_excel(fp, sheet_name=['LK_7-Tage-Inzidenz', 'LK_7-Tage-Fallzahlen'], header=None,
                       na_filter=False, engine="openpyxl")

nan_value = float("NaN")

data = df['LK_7-Tage-Inzidenz'].iloc[4:,2:].replace("", nan_value).dropna(how='all', axis=1)

data
parse_and_insert_sheet(df, 'LK_7-Tage-Inzidenz', 'egal')

INFO:__main__:Parse sheet LK_7-Tage-Inzidenz


5334
7131
9771
8425
16077
7132
15081
9171
7331
9361
9371
3451
15082
9561
9571
9661
9671
9761
9772
3452
7332
9672
7133
9173
8211
9461
9471
12060
14625
9462
9472
9172
6431
11004
11002
11011
11010
11001
11008
11003
11012
11005
11006
11007
11009
7231
8426
5711
7134
7232
8115
5911
8435
5314
15083
5554
5512
12051
3101
8315
4011
4012
15084
8235
3351
9372
14511
3453
9463
9473
7135
5558
12052
3352
9174
12061
6411
6432
9271
3401
15001
3251
9773
9279
1051
9779
7333
5913
14612
5112
5358
5111
9175
16061
9176
16056
12062
3402
8316
3454
5954
8236
9177
16051
9562
9572
14521
5113
8116
5366
1001
9474
7311
12053
6412
8311
9178
8237
9272
3455
6631
9179
9563
9573
9180
5513
16052
7334
6531
3151
8117
14626
3153
16067
3159
3456
16076
6433
9774
5754
5914
15002
2000
3252
5915
3241
3353
15085
9674
12063
3358
8221
8135
8121
8125
5370
3154
5758
5916
6632
1053
16069
3254
5958
6434
9464
9475
8126
3255
5762
16070
9161
16053
15086
7312
7335
8212
8215
6611
6633
9762
9273
9763
1002
9675
5154
7111
5315
8335
5114
9476
947

[{'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '05334',
  'ts': datetime.datetime(2020, 11, 18, 0, 0),
  'val': 211.839303730885},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '05334',
  'ts': datetime.datetime(2020, 11, 19, 0, 0),
  'val': 179.345308836571},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '05334',
  'ts': datetime.datetime(2020, 11, 20, 0, 0),
  'val': 174.318613493805},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '05334',
  'ts': datetime.datetime(2020, 11, 21, 0, 0),
  'val': 161.392825469547},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '05334',
  'ts': datetime.datetime(2020, 11, 22, 0, 0),
  'val': 121.179262727413},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '05334',
  'ts': datetime.datetime(2020, 11, 23, 0, 0),
  'val': 115.973042550976},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '05334',
  'ts': dat